In [4]:
# %%capture
# !pip install selenium

## 지금까지 나온 문제
    대문 img를 말고 하단에 img가 추가 되어있는 책과 아닌책이 있어서 한칸씩 밀리는 경우
    책 속으로 라고하는 챕터가 있거나 없거나

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [36]:
driver = webdriver.Chrome()
# driver.get('https://product.kyobobook.co.kr/new/#?page=1&sort=new&year=2023&month=04&week=3&per=20&saleCmdtDvsnCode=KOR&gubun=newGubun&saleCmdtClstCode=')
# driver.get('https://product.kyobobook.co.kr/detail/S000201423559')
driver.get('https://product.kyobobook.co.kr/detail/S000201470540')

In [37]:
#책소개 
#위에 이미지가 있을경우 한칸씩 밀려서 작성된다?
print(driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]').get_attribute('class'))

product_detail_area detail_img


In [ ]:
def img_finder():
    #product_detail_area detail_img / product_detail_area book_intro
    standard = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]').get_attribute('class')
    if standard == 'product_detail_area detail_img':
        pass
    elif standard == 'product_detail_area book_intro':
        pass
    else:
        return print('예외처리가 안된 XPATH')
        


In [38]:
title = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[1]/div/div[1]/div[1]/div/h1/span').text
title

'아웃풋 법칙'

In [39]:
sub_title = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[1]/div/div[3]/div/div[1]/div/span').text
sub_title

'평범한 사람도 압도적 성공으로 이끈 단 하나의 원리'

In [41]:
author = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div/a').text
# author = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[1]/div/div/a').text
# //*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[1]/div/div/a img 있는 url
#//*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div/a img 없는 url
author

'김재수(렘군)'

In [42]:
price = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[3]/div[1]/div[2]/div').text
price

'10% 16,920원 18,800원'

In [43]:
img = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[2]/div[1]/div[1]/ul/li[1]/div/div[2]/img').get_attribute('src')
img

'https://contents.kyobobook.co.kr/sih/fit-in/458x0/pdt/9791140704033.jpg'

In [45]:
cls = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]/div[1]/ul/li/a[2]').text
# cls = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[5]/div[1]/ul/li/a[2]').text
# //*[@id="scrollSpyProdInfo"]/div[5]/div[1]/ul/li/a[2] img 있는 url
# //*[@id="scrollSpyProdInfo"]/div[4]/div[1]/ul/li/a[2] 없는 url
cls

'자기계발'

In [48]:
# intro = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]/div[2]/div').text
intro = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[5]/div[2]').text
intro

'구독자 32만 명의 유튜브 〈푸릉_렘군〉 대표, 대한민국 최고의 부동산 전문가,\n10만 베스트셀러 작가, 3040 직장인들의 멘토,\n렘군의 첫 자기계발서 출간!\n어느덧 직장경력 10여 년, 항상 최선을 다해 열심히 살아온 것 같은데 생각보다 손에 쥐고 있는 것이 별로 없다고 느끼는 사람들이 꽤 많다. 자기계발에 소홀한 것도 아니고, 책도 꽤 읽고 공부도 한 것 같은데 왜 이렇게 회사를 벗어나서는 할 줄 아는 게 없다고 느껴지는 걸까? 지난 5년간 렘군은 ‘시작 컨설팅’이라는 이름으로 평범한 사람들이 자신의 강점을 찾고 제2의 인생 설계를 할 수 있도록 도왔다. 그는 이 과정을 통해 얻은 지식을 ‘아웃풋(output)’이라는 단어로 설명한다. 아웃풋은 단순히 어떤 생산물 또는 결과물만을 의미하지 않는다. 저자는 이를 타인을 위해, 세상과 연결되기 위해 제공하는 생산물로 정리한다.\n우리는 늘 자신이 완벽하지 않다고 생각해 강의를 듣고, 책을 읽으며 인풋만 일삼는다. 남들이 만들어 놓은 지식, 콘텐츠, 물건 등을 소비하는 데 급급해 자신이 무엇을 세상에 제공할 수 있는지 잊고 산다. 그러나 조금 부족하더라도, 완벽하지 않더라도 자신의 아웃풋을 세상에 내놓는 바로 그 순간, ‘소비자의 영역’에서 벗어나 ‘생산자의 영역’으로 진입하게 된다. 이때 비로소 삶에 변화가 일어나는 것이다. 이렇게 저자는 ‘아웃풋 법칙’을 6단계로 정리해 사람들이 쉽고 간단하게 생산자의 삶을 살 수 있도록 안내한다.\n\n아웃풋 법칙 1단계. 피라미드 밖으로 뛰쳐나가라.\n아웃풋 법칙 2단계. 성공의 사분면을 찾아라.\n아웃풋 법칙 3단계. 나만의 정체성을 발견하라.\n아웃풋 법칙 4단계. 세상을 향해 아웃풋하라.\n아웃풋 법칙 5단계. 넘버원이 아닌 온리원이 돼라.\n아웃풋 법칙 6단계. 저항을 완전히 무력화시켜라.\n\n친절하면서도 예리한 저자의 관점을 즐기면서 차근차근 실행하다 보면, 어느새 나만의 아웃풋을 만들어내는 자신을 발견할 수 있을 것이다. 이 일은 지금까지 습관과 관성

In [50]:
# contents = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[8]/div[2]/div[1]/div/ul/li').text
contents = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[9]/div[2]/div[1]/div/ul/li').text
contents


"들어가며 소비자의 삶에서 생산자의 삶으로\n\n1단계. 피라미드 밖으로 뛰쳐나가라\n나는 어떻게 101가지 직업을 가질 수 있을까\n경쟁만이 가득한 피라미드 세계\n당신이 바라보는 세상은 어떤 모습인가\n피라미드 바깥세상을 바라보라\n\n2단계. 성공의 사분면을 찾아라\n열정보다 방향성이 중요하다\n자신만의 기준으로 세상을 정의한 사람들\n개인이 곧 기업인 시대의 성공 법칙\n성공으로 가는 네 가지 길\n네 가지 중 무엇을 선택할 것인가\nMCBI를 내 삶에 적용하는 법\n그들도 이루었듯이, 당신도 할 수 있다\n\n3단계. 나만의 정체성을 발견하라\n정체성이란 무엇인가\n정체성의 비밀 1 - 자기소개에 숨겨진 진실\n정체성의 비밀 2 - 나를 가로막고 있는 것은 무엇인가\n정체성의 비밀 3 - 결핍을 원동력으로 바꾸기\n정체성의 비밀 4 - '미래의 나'를 따라가라\n정체성의 비밀 5 - 나만 모르는 나의 강점\n정체성의 비밀 6 - 나를 찾는 매트릭스\n정체성의 비밀 7 - 자기표현과 피드백\n정체성의 비밀 8 - '좋아한다'의 오류\n정체성의 비밀 9 - 나 찾기 프로젝트 5단계\n세상에 나의 발자국을 남겨라\n\n4단계. 세상을 향해 아웃풋하라\n아웃풋이란 무엇인가\n아웃풋 법칙 1 - 제로에서 제로답게 시작하는 법\n아웃풋 법칙 2 - 정체성을 키워드로 연결시키는 법\n아웃풋 법칙 3 - 딱 한 사람의 마음을 사로잡는 법\n아웃풋 법칙 4 - 나를 각인시키는 법\n아웃풋 법칙 5 - 빠른 시작의 기술\n아웃풋 법칙 6 - MCBI에 맞는 아웃풋 전략\n\n5단계. 넘버원이 아닌 온리원이 돼라\n넘버원은 허상이다\n온리원의 법칙 1 - 지금 하는 일의 '그다음'을 먼저 생각하라\n온리원의 법칙 2 - 하나를 잘게 쪼개 새로운 일을 발견하라\n온리원의 법칙 3 - 키워드 하나에서 시작해 좁히고 비틀어라\n온리원의 법칙 4 - '기브 앤드 테이크' 말고 기브, 기브, 기브 하라\n온리원의 법칙 5 - 당신의 기적을 노래하라\n온리원의 법칙 6 - 생태계를

In [55]:
inner_book = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[10]/div[2]/div[1]/div/p').text
inner_book

'전체 세상의 90퍼센트를 차지하는 바깥세상, 그곳에서의 경쟁은 피라미드 안보다 여유롭다. 위로 올라가지 않아도, 조금만 방향을 바꿔도 얼마든지 풍요롭게 먹고살 수 있는 길이 보인다. 그러다 보니 내가 무엇을 좋아하는지, 타인이 무엇을 원하는지 살펴볼 여유시간도 생긴다. 피라미드 밖에 터를 잡고 피라미드 안팎을 자유롭게 오가며 세상 사람들이 원하는 것을 끊임없이 제공한다. 인생을 주체적이고 풍요롭게 살고 싶다면 작은 삼각형 안에 머무르는 게 맞을까, 바깥세상으로 뛰쳐나가는 게 맞을까? 정답은 없다. 모든 사람이 바깥세상으로 나올 필요도 없을 것이다. 하지만 현재의 삶이 불만족스럽거나 지금과는 전혀 다른 인생을 살고 싶은 사람들은 대부분 피라미드 바깥세상을 선택했다. 당신이 살고 싶은 세상의 모습은 어떤지 생각해보자. 혹시 피라미드만을 바라보고 있는가? 그렇다면 평생 끝나지 않는 경쟁 속에서 살아야 할 것이다. 세상의 모습에 대해 제대로 생각해본 적이 없다면 재정립해야 한다.\n_ 1단계: 피라미드 바깥으로 뛰쳐나가라\n\n일단, ‘성공의 방향성’이 있다고 믿어보라. 그 길을 걸어간 사람들의 공통점을 찾아보라. 공통점의 핵심을 꿰뚫어보라. 단순히 어떤 아이디어, 어떤 제품, 어떤 방식, 어떤 시대, 어떤 배경이 중요한 게 아니다. 올바른 길이 어느 쪽인지 방향을 설정하는 것이 무엇보다 중요하다. 그다음 최선을 다했을 때 원하는 곳에 도착할 수 있다.\n_2단계: 성공의 사분면을 찾아라\n\n정체성은 하루아침에 갑자기 나타나는 것이 아니다. 정체성은 모든 가능성을 생각하고, 그걸 행동으로 옮기면서 ‘나도 이런 걸 할 수 있구나, 사람들은 이런 걸 원하는구나’ 하며 깨달아가는 것이다. 정체성은 나 혼자 만드는 게 아니다. 정체성은 타인의 피드백과 함께 만들어지는 것이다. 정체성은 사람들이 나를 인식하는 모습이다. 나의 본질적인 성향과 연관이 있지만 오직 나의 생각만으로 완성되는 것은 아니다.\n_3단계: 나만의 정체성을 발견하라\n\n아웃풋의 방향은 내가 아닌 타인

In [53]:
# publisher_review = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[9]/div[2]/div[1]/div/p').text
publisher_review = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[11]/div[2]/div[1]/div/p').text
publisher_review


'“세상은 소비하는 사람이 아니라 뭔가를 만들어내는 사람이 바꾼다!”\n경제적 자유로 가는 확실하고 구체적인 실전 로드맵\n\n나름대로 열심히, 바쁘게 살아온 것 같은데 어느덧 직장경력 10여 년, 생각보다 손에 쥐고 있는 것이 별로 없다고 느끼는 사람들이 꽤 많다. 쌓이는 연차만큼 연봉도 올라가지만 갚아야 할 아파트 대출금은 쉽게 줄어들지 않고, 이제는 직급이 낮진 않지만 회사 밖에서 써먹을 만한 자기 브랜드도, 전문성도 가지고 있는 것 같지 않다. 자기계발에 소홀한 것도 아니고, 책도 꽤 읽고 공부도 한 것 같은데 왜 이렇게 회사를 벗어나서는 할 줄 아는 게 없다고 느껴지는 걸까?\n정년은커녕 5년 후도 불안한 지금, 3040에게 가장 필요한 능력은 무엇일까? 데이터를 이용한 부동산 투자로 유명한 재테크 전문가 렘군은 그의 첫 자기계발서 《아웃풋 법칙》을 통해 말한다. 이제 그만 좀 공부하고 제발 그 지식을 세상 밖으로 내놓으라고.\n\n지난 5년간 렘군은 ‘시작 컨설팅’이라는 이름으로 평범한 사람들이 자신의 강점을 찾고 제2의 인생 설계를 할 수 있도록 도왔다. 그는 이 과정을 통해 얻은 지식을 ‘아웃풋(output)’이라는 단어로 설명한다. 아웃풋은 단순히 어떤 생산물 또는 결과물만을 의미하지 않는다. 저자는 이를 타인을 위해, 세상과 연결되기 위해 제공하는 생산물로 정리한다.\n많은 사람이 ‘나는 부족해’ ‘아직 내 지식은 세상에 내놓을 것이 못 돼’ ‘좀 더 공부해야 해’ 하면서 더 훌륭해진 후에야 뭔가를 하려고 한다. 그러나 그 시기는 절대 오지 않는다고 저자는 잘라 말한다. 혼자서 최고의 완성도를 올리기 위해 노력하기보다는 끊임없이 세상에 무엇인가를 내놓으면서 ‘타인 혹은 세상이 원하는 것’이 무엇인지를 찾으라고 강조한다. 완벽하지 않더라도 자신의 아웃풋을 세상에 내놓는 바로 그 순간, ‘소비자의 영역’에서 벗어나 ‘생산자의 영역’으로 진입하게 되고, 그제야 비로소 삶에 변화가 일어난다는 사실을 알려준다.\n\n“언제까지 미련하게 인풋만 할 텐가

In [57]:
# ISBN = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[10]/div[2]/table/tbody/tr[1]/td').text
ISBN = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[12]/div[2]/table/tbody/tr[1]/td').text
ISBN

'9791140704033'

### 책마다 XPATH가 달라요..

In [5]:
driver.get('https://product.kyobobook.co.kr/detail/S000201470540')

<selenium.webdriver.remote.webelement.WebElement (session="3787e6e269ddb79403397ca97154aa80", element="b209b8aa-b541-495b-ae08-b0871f515eda")>

In [13]:
driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[5]/div[2]').text


'구독자 32만 명의 유튜브 〈푸릉_렘군〉 대표, 대한민국 최고의 부동산 전문가,\n10만 베스트셀러 작가, 3040 직장인들의 멘토,\n렘군의 첫 자기계발서 출간!\n어느덧 직장경력 10여 년, 항상 최선을 다해 열심히 살아온 것 같은데 생각보다 손에 쥐고 있는 것이 별로 없다고 느끼는 사람들이 꽤 많다. 자기계발에 소홀한 것도 아니고, 책도 꽤 읽고 공부도 한 것 같은데 왜 이렇게 회사를 벗어나서는 할 줄 아는 게 없다고 느껴지는 걸까? 지난 5년간 렘군은 ‘시작 컨설팅’이라는 이름으로 평범한 사람들이 자신의 강점을 찾고 제2의 인생 설계를 할 수 있도록 도왔다. 그는 이 과정을 통해 얻은 지식을 ‘아웃풋(output)’이라는 단어로 설명한다. 아웃풋은 단순히 어떤 생산물 또는 결과물만을 의미하지 않는다. 저자는 이를 타인을 위해, 세상과 연결되기 위해 제공하는 생산물로 정리한다.\n우리는 늘 자신이 완벽하지 않다고 생각해 강의를 듣고, 책을 읽으며 인풋만 일삼는다. 남들이 만들어 놓은 지식, 콘텐츠, 물건 등을 소비하는 데 급급해 자신이 무엇을 세상에 제공할 수 있는지 잊고 산다. 그러나 조금 부족하더라도, 완벽하지 않더라도 자신의 아웃풋을 세상에 내놓는 바로 그 순간, ‘소비자의 영역’에서 벗어나 ‘생산자의 영역’으로 진입하게 된다. 이때 비로소 삶에 변화가 일어나는 것이다. 이렇게 저자는 ‘아웃풋 법칙’을 6단계로 정리해 사람들이 쉽고 간단하게 생산자의 삶을 살 수 있도록 안내한다.\n\n아웃풋 법칙 1단계. 피라미드 밖으로 뛰쳐나가라.\n아웃풋 법칙 2단계. 성공의 사분면을 찾아라.\n아웃풋 법칙 3단계. 나만의 정체성을 발견하라.\n아웃풋 법칙 4단계. 세상을 향해 아웃풋하라.\n아웃풋 법칙 5단계. 넘버원이 아닌 온리원이 돼라.\n아웃풋 법칙 6단계. 저항을 완전히 무력화시켜라.\n\n친절하면서도 예리한 저자의 관점을 즐기면서 차근차근 실행하다 보면, 어느새 나만의 아웃풋을 만들어내는 자신을 발견할 수 있을 것이다. 이 일은 지금까지 습관과 관성

In [23]:
print(driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]').get_attribute('class'))

product_detail_area detail_img


In [30]:
driver.get('https://product.kyobobook.co.kr/detail/S000201505566')

In [31]:
print(driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]').get_attribute('class'))

product_detail_area book_intro
